In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


2021-07-13 19:12:16.089116: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
df = pd.read_csv("Train_Data.csv")
test=pd.read_csv("Test_Data.csv")

In [3]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
le = LabelEncoder()
# lb = LabelBinarizer()
df['adgroup'] = le.fit_transform(df['adgroup'])
test['adgroup']=le.transform(test['adgroup'])
# df = df.join(pd.DataFrame(lb.fit_transform(df['adgroup'])))
# test = test.join(pd.DataFrame(lb.transform(test['adgroup'])))
# df.drop(['adgroup'],axis=1)
# test.drop(['adgroup'],axis=1)
df = df.set_index(df['date'])
test = test.set_index(test['date'])

df.drop(['date', 'campaign'],axis=1,inplace=True)
test.drop(['date','campaign'],axis=1,inplace=True)


In [4]:
for i in range(len(df)):
    df['ad'][i] = int(df['ad'][i][3:])
for i in range(len(test)):
    test['ad'][i]=int(test['ad'][i][3:])
df['ad']=pd.to_numeric(df['ad'])
test['ad']=pd.to_numeric(test['ad'])
df.drop(['ad'],axis=1,inplace=True)

/tmp/ipykernel_24038/2739739156.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ad'][i] = int(df['ad'][i][3:])
/tmp/ipykernel_24038/2739739156.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['ad'][i]=int(test['ad'][i][3:])


In [5]:
df['CTR']=df['clicks']/df['impressions']
df['CPC']=df['cost']/df['clicks']
df['CPA']=df['cost']/df['conversions']

test['CTR']=test['clicks']/test['impressions']
test['CPC']=test['cost']/test['clicks']
test['CPA']=test['cost']/test['conversions']

df['CPC'].fillna(0,inplace=True)
df['CPA'].fillna(0,inplace=True)

test['CPC'].fillna(0,inplace=True)
test['CPA'].fillna(0,inplace=True)

df.replace([np.inf], 65,inplace=True)
test.replace([np.inf], 65,inplace=True)

In [6]:
# y = df['revenue']
# df.drop(['revenue','campaign'], axis=1,inplace=True)
# test=np.array(test)
# df = np.array(df)

In [7]:
# df = np.array(df)
x_train, x_test = train_test_split(df)

In [8]:
from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.layers import Dense, Dropout, LSTM
# from tensorflow.keras.models import Sequential

In [9]:
# from tensorflow.keras import backend
# import tensorflow as tf
 
# def rmse(y_true, y_pred):
# 	return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [10]:
# # x_train=np.array(x_train).reshape(x_train.shape[0],x_train.shape[1],1)
# model = Sequential()                # could make units 50, for better prediction
# # model.add(Dense(units=40,activation='selu'))
# model.add(LSTM(units=40,activation='tanh',input_shape=(x_train.shape[1],1)))
# model.add(Dropout(0.4))
# model.add(Dense(units=40,activation='selu'))
# model.add(Dropout(0.4))
# # model.add(LSTM(units=40,activation='tanh'))
# # model.add(Dense(units=30,activation='relu'))
# model.add(Dense(units=30,activation='relu'))
# model.add(Dropout(0.4))
# model.add(Dense(units=40,activation='selu'))
# model.add(Dropout(0.4))
# model.add(Dense(units=40,activation='selu'))

# model.add(Dense(1,activation='relu'))
# model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='mean_squared_error',metrics=[rmse])
# model.fit(x_train,y_train, epochs=30, batch_size=60)

In [11]:
# m1 = RandomForestRegressor()
# m1.fit(x_train,y_train)
# x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],1)
# y_pred = model.predict(x_test)
# rms = mean_squared_error(y_test, y_pred, squared=False)
# rms

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRFRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error
from xgboost import XGBRFRegressor, XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
import catboost as ctb

In [20]:
g = df.groupby('adgroup')

# model0 = XGBRFRegressor(n_estimators=10,max_depth=6, min_child_weight=2,subsample=0,learning_rate = 0.3)
# model0 = lg(kernel='rbf', degree=2, gamma='auto', C = 0.1)
j=6
model0 = ctb.CatBoostRegressor(random_state=0,verbose=False)
model1 = ctb.CatBoostRegressor(random_state=0,verbose=False)
model2 = ctb.CatBoostRegressor(random_state=0,verbose=False)
model3 = ctb.CatBoostRegressor(random_state=0,verbose=False)
for i in range(4):
    d=pd.DataFrame(g.get_group(i))
    y=d['revenue']
    # print(d.head(5))
    d.drop(['revenue'],axis=1,inplace=True)
    d_train,d_test,y1_train,y1_test = train_test_split(d,y,random_state=0)
    grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}
    if(i==0):
        # model0.fit(d_train,y1_train)
        grid_search_result = model0.grid_search(grid, 
                                       X=d_train, 
                                       y=y1_train, 
                                       verbose=False,refit=True, search_by_train_test_split=True)
        y_pred=model0.predict(d_test)
        
    elif (i==1):
        # model1.fit(d_train,y1_train)
        grid_search_result = model1.grid_search(grid, 
                                       X=d_train, 
                                       y=y1_train, 
                                       verbose=False,refit=True, search_by_train_test_split=True)
        y_pred=model1.predict(d_test)
    elif (i==2):
        # model2.fit(d_train,y1_train)
        grid_search_result = model2.grid_search(grid, 
                                       X=d_train, 
                                       y=y1_train, 
                                       verbose=False,refit=True, search_by_train_test_split=True)
        y_pred=model2.predict(d_test)
    elif (i==3):
        # model3.fit(d_train,y1_train)
        grid_search_result = model3.grid_search(grid, 
                                       X=d_train, 
                                       y=y1_train, 
                                       verbose=False,refit=True, search_by_train_test_split=True) 
        y_pred=model3.predict(d_test)
    
    for i in range(len(d_test)):
        if(y_pred[i]<0):
            y_pred[i] = 0
    rms = mean_squared_error(y1_test, y_pred, squared=False)
    print(rms)



bestTest = 759.0409008
bestIteration = 968


bestTest = 782.9209095
bestIteration = 253


bestTest = 754.0498971
bestIteration = 763


bestTest = 749.3737281
bestIteration = 102


bestTest = 758.4589821
bestIteration = 935


bestTest = 746.9467017
bestIteration = 541


bestTest = 758.6290306
bestIteration = 999


bestTest = 767.8791423
bestIteration = 446


bestTest = 761.2243433
bestIteration = 857


bestTest = 712.8884736
bestIteration = 205


bestTest = 827.5405371
bestIteration = 528


bestTest = 900.7586076
bestIteration = 161


bestTest = 918.4055241
bestIteration = 762


bestTest = 934.5313146
bestIteration = 106


bestTest = 906.9067868
bestIteration = 977


bestTest = 895.2887338
bestIteration = 265


bestTest = 899.2499555
bestIteration = 948


bestTest = 856.4458183
bestIteration = 407


bestTest = 911.5434681
bestIteration = 949


bestTest = 871.3440056
bestIteration = 278


bestTest = 985.8511043
bestIteration = 728


bestTest = 936.944517
bestIteration = 110


bestTest =

In [21]:
y_pred=[]
# x_test.drop('revenue',axis=1,inplace=True)
x_test = test
for i in range(len(x_test)):
    if(x_test['adgroup'][i]==0):
        y_pred = np.append(y_pred,model0.predict(pd.DataFrame(x_test.iloc[i]).transpose())[0])
    elif (x_test['adgroup'][i]==1):
        y_pred = np.append(y_pred,model1.predict(pd.DataFrame(x_test.iloc[i]).transpose())[0])
    elif (x_test['adgroup'][i]==2):
        y_pred = np.append(y_pred,model2.predict(pd.DataFrame(x_test.iloc[i]).transpose())[0])
    elif (x_test['adgroup'][i]==3):
        y_pred = np.append(y_pred,model3.predict(pd.DataFrame(x_test.iloc[i]).transpose())[0])




In [67]:
y_pred.shape

(318,)

In [ ]:
np.array(x_test.iloc[i,:]).reshape(1,-1)

In [ ]:
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [10],
        'min_child_weight': [ 5],
        # 'subsample': [0.5, 0.7],
        # 'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [200],
        'objective': ['reg:squarederror']
    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

In [ ]:
y_pred[0]

In [ ]:
## Final
model2.fit(df,y)
y_final = model2.predict(test)
for i in range(len(y_final)):
    if y_final[i] < 0:
        y_final[i]=0

In [68]:
pd.DataFrame({'revenue':y_pred}).to_csv("submission.csv")

In [ ]:
sns.scatterplot(df['adgroup'],df['revenue'])

In [20]:
pd.DataFrame({'revenue':yf['Label']}).to_csv("submission3.csv")